In [1]:
import os
import cv2
import shutil
import numpy as np
import skimage
from skimage import io
import argparse
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms

In [23]:
import torch.nn.functional as F

In [16]:
from collections import OrderedDict

In [21]:
import math

In [6]:
import matplotlib.pyplot as plt

In [70]:
imgPath = 'Wrap.jpg'
modelPath = 'model_geoNet.pkl'
saveImgPath = 'result.jpg'
saveFlowPath = 'IMG_WRAP.npy'

In [18]:
class plainEncoderBlock(nn.Module):
    def __init__(self, inChannel, outChannel, stride):
    
        super(plainEncoderBlock, self).__init__()
        self.conv1 = nn.Conv2d(inChannel, outChannel, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(outChannel)
        self.conv2 = nn.Conv2d(outChannel, outChannel, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(outChannel)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        return x
    
class plainDecoderBlock(nn.Module):
    def __init__(self, inChannel, outChannel, stride):
    
        super(plainDecoderBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(inChannel, inChannel, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(inChannel)
        
        self.conv2 = nn.Conv2d(inChannel, outChannel, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(outChannel)
        
        self.up = None
        
        if stride != 1:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear')
            #self.up = nn.Upsample(scale_factor=2, mode='nearest')
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        
        if self.up is not None:
            x = self.up(x)
        
        return x
    

class resEncoderBlock(nn.Module):
    def __init__(self, inChannel, outChannel, stride):
    
        super(resEncoderBlock, self).__init__()
        self.conv1 = nn.Conv2d(inChannel, outChannel, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(outChannel)
        self.conv2 = nn.Conv2d(outChannel, outChannel, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(outChannel)
        
        self.downsample = None
        if stride != 1:  
            self.downsample = nn.Sequential(
                nn.Conv2d(inChannel, outChannel, kernel_size=1, stride=stride),
                nn.BatchNorm2d(outChannel))
        
        
    def forward(self, x):
        residual = x
        
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        if self.downsample is not None:
            residual = self.downsample(x)
        
        out += residual
        out = F.relu(out)
        return out
    
class resDecoderBlock(nn.Module):
    def __init__(self, inChannel, outChannel, stride):
    
        super(resDecoderBlock, self).__init__()
        self.conv1 = nn.Conv2d(inChannel, inChannel, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(inChannel)
        
        self.downsample = None
        self.up = None
        
        if stride == 1:
            self.conv2 = nn.Conv2d(inChannel, outChannel, kernel_size=3, stride=1, padding=1)
            self.bn2 = nn.BatchNorm2d(outChannel)
        else:
            self.conv2 = nn.Conv2d(inChannel, outChannel, kernel_size=3, stride=1, padding=1)
            self.bn2 = nn.BatchNorm2d(outChannel)
            self.up = nn.Upsample(scale_factor=2, mode='bilinear')
            #self.up = nn.Upsample(scale_factor=2, mode='nearest')
            
            self.downsample = nn.Sequential(
                nn.Conv2d(inChannel, outChannel, kernel_size=1, stride=1),
                nn.BatchNorm2d(outChannel),
                nn.Upsample(scale_factor=2, mode='bilinear'))
                #nn.Upsample(scale_factor=2, mode='nearest'))   
        
    def forward(self, x):
        residual = x
        
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        if self.up is not None:
            out = self.up(out)

        if self.downsample is not None:
            residual = self.downsample(x)
        
        out += residual
        out = F.relu(out)
        return out

In [19]:
class GeoNet(nn.Module):
    def __init__(self, layers):
        super(GeoNet, self).__init__()  
        
        self.conv = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(64)
        
        self.en_layer1 = self.make_encoder_layer(plainEncoderBlock, 64, 64, layers[0], stride=1)  
        self.en_layer2 = self.make_encoder_layer(resEncoderBlock, 64, 128, layers[1], stride=2)
        self.en_layer3 = self.make_encoder_layer(resEncoderBlock, 128, 256, layers[2], stride=2)
        self.en_layer4 = self.make_encoder_layer(resEncoderBlock, 256, 512, layers[3], stride=2)
        self.en_layer5 = self.make_encoder_layer(resEncoderBlock, 512, 512, layers[4], stride=2)
        
        self.de_layer5 = self.make_decoder_layer(resDecoderBlock, 512, 512, layers[4], stride=2)
        self.de_layer4 = self.make_decoder_layer(resDecoderBlock, 512, 256, layers[3], stride=2)
        self.de_layer3 = self.make_decoder_layer(resDecoderBlock, 256, 128, layers[2], stride=2)
        self.de_layer2 = self.make_decoder_layer(resDecoderBlock, 128, 64, layers[1], stride=2)
        self.de_layer1 = self.make_decoder_layer(plainDecoderBlock, 64, 64, layers[0], stride=1)
        
        self.conv_end = nn.Conv2d(64, 2, kernel_size=3, stride=1, padding=1)
        
        self.fconv = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.fbn = nn.BatchNorm2d(64)
        self.f_en_layer1 = self.make_encoder_layer(plainEncoderBlock, 64, 64, layers[0], stride=1)  
        self.f_en_layer2 = self.make_encoder_layer(resEncoderBlock, 64, 128, layers[1], stride=2)
        self.f_en_layer3 = self.make_encoder_layer(resEncoderBlock, 128, 256, layers[2], stride=2)
        self.f_en_layer4 = self.make_encoder_layer(resEncoderBlock, 256, 512, layers[3], stride=2)
        self.f_en_layer5 = self.make_encoder_layer(resEncoderBlock, 512, 512, layers[4], stride=2)
        
        self.f_en_layer6 = self.make_encoder_layer(resEncoderBlock, 512, 512, 1, stride=2)
        self.f_en_layer7 = self.make_encoder_layer(resEncoderBlock, 512, 512, 1, stride=2)
        self.f_en_layer8 = self.make_encoder_layer(resEncoderBlock, 512, 512, 1, stride=2)
        
        self.fc1 = nn.Linear(512 * 2 * 2, 1024)
        self.fc1bn = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc2bn = nn.BatchNorm1d(512)
        
        self.catconv = nn.Conv2d(1024, 512, kernel_size=1, bias=False)
        self.catbn  = nn.BatchNorm2d(512)
        
                       
        # weight initializaion with Kaiming method
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
        
    def make_encoder_layer(self, block, inChannel, outChannel, block_num, stride):

        layers = []
        layers.append(block(inChannel, outChannel, stride=stride))
        for i in range(1, block_num):
            layers.append(block(outChannel, outChannel, stride=1))

        return nn.Sequential(*layers)
    
    def make_decoder_layer(self, block, inChannel, outChannel, block_num, stride):

        layers = []
        for i in range(0, block_num-1):
            layers.append(block(inChannel, inChannel, stride=1))
            
        layers.append(block(inChannel, outChannel, stride=stride))
        
        return nn.Sequential(*layers)
                       
    def forward(self, x, y):
        
        x = F.relu(self.bn(self.conv(x)))
        x = self.en_layer1(x)     #256
        x = self.en_layer2(x)     #128
        x = self.en_layer3(x)     #64
        x = self.en_layer4(x)     #32
        x = self.en_layer5(x)     #16
        
        y = F.relu(self.fbn(self.fconv(y)))
        y = self.f_en_layer1(y)     #256
        y = self.f_en_layer2(y)     #128
        y = self.f_en_layer3(y)     #64
        y = self.f_en_layer4(y)     #32
        y = self.f_en_layer5(y)     #16
        
        y = self.f_en_layer6(y)     #8
        y = self.f_en_layer7(y)     #4
        y = self.f_en_layer8(y)     #2
        
        y = y.view(-1, 512*2*2)
        
        y = F.relu(self.fc1bn(self.fc1(y)))
        y = F.relu(self.fc2bn(self.fc2(y)))
        y = y.unsqueeze(2).unsqueeze(2).expand_as(x)
        
        x = torch.cat([x, y], dim=1)
        
        x = F.relu(self.catbn(self.catconv(x)))
        
        x = self.de_layer5(x)     
        x = self.de_layer4(x)     
        x = self.de_layer3(x)     
        x = self.de_layer2(x)     
        x = self.de_layer1(x)        
        
        x = self.conv_end(x)
        return x

### Resizing the image

In [71]:
H = 2000
W = 1500
img = io.imread(imgPath)
h, w = img.shape[0:2]
    
if h > w:
    ratio = float(h)/float(w)

    if (ratio > float(H)/float(W)):
        img = skimage.transform.resize(img, (int(ratio*W), W), order=1)
    else:
        img = skimage.transform.resize(img, (H, int(H/ratio)), order=1)

    yc = int(img.shape[0]/2)
    xc = int(img.shape[1]/2)
    img = img[yc - int(H/2):yc + int(H/2), xc - int(W/2):xc + int(W/2)]
        
else:
    ratio = float(w)/float(h)
        
    if (ratio > float(H)/float(W)):
        img = skimage.transform.resize(img, (W, int(W*ratio)), order=1)
    else:
        img = skimage.transform.resize(img, (int(H/ratio), H), order=1)
         
    yc = int(img.shape[0]/2)
    xc = int(img.shape[1]/2)
    img = img[yc - int(W/2):yc + int(W/2), xc - int(H/2):xc + int(H/2)]

In [72]:
img.shape

(2000, 1500, 3)

### Image Padding

In [9]:
def padImg(img):
    '''
    pad image twice.
    The first padding is to make sure the patches cover all image regions.
    The second padding is used for cropping the global patch.
    '''
    
    H = img.shape[0]
    W = img.shape[1]
    
    globalFct = 4
    patchRes = 256
    ovlp = int(patchRes * 0.25)
    
    padH = (int((H - patchRes)/(patchRes - ovlp) + 1) * (patchRes - ovlp) + patchRes) - H
    padW = (int((W - patchRes)/(patchRes - ovlp) + 1) * (patchRes - ovlp) + patchRes) - W
    
    padding = int(patchRes * (globalFct - 1) / 2.0)

    padImg = cv2.copyMakeBorder(img, 0, padH, 0, padW, cv2.BORDER_REPLICATE)
    padImg = cv2.copyMakeBorder(padImg, padding, padding, padding, padding, cv2.BORDER_REPLICATE)
    
    return padImg

In [10]:
def cropToPatch(img):
    '''
    crop the image to local and global patches
    '''

    H = img.shape[0]
    W = img.shape[1]
    
    globalFct = 4
    patchRes = 256
    ovlp = int(patchRes * 0.25)
    padding = int(patchRes * (globalFct - 1) / 2.0)

    cropH = patchRes
    cropW = patchRes

    ynum = int((H - (globalFct - 1) * cropH - cropH)/(cropH - ovlp)) + 1
    xnum = int((W - (globalFct - 1) * cropW - cropW)/(cropW - ovlp)) + 1
    
    totalLocal = np.zeros((ynum, xnum, patchRes, patchRes, 3), dtype=np.uint8)
    totalGloba = np.zeros((ynum, xnum, 256, 256, 3), dtype=np.uint8)

    for j in range(0, ynum):
        for i in range(0, xnum):

            x = int(padding + i * (cropW - ovlp))
            y = int(padding + j * (cropH - ovlp))

            totalLocal[j, i] = img[y:int(y + patchRes), x:int(x + patchRes)]

            gx = int(x - padding)
            gy = int(y - padding)
            globalpatch = img[gy:int(gy + globalFct * patchRes), gx:int(gx + globalFct * patchRes)]
            globalpatch = skimage.transform.resize(globalpatch, (256, 256)) * 255.0
            totalGloba[j, i] = globalpatch
            
    return totalLocal, totalGloba

In [73]:
io.imsave(saveImgPath, img)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [74]:
totalLocalPatch, totalGlobaPatch = cropToPatch(img)

In [13]:
def testRealFlow(modelPath, localPatch, globalPatch):
    '''
    estimate the flows
    '''

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    model = GeoNet([1, 1, 1, 1, 1])

    if torch.cuda.is_available():
        model = model.cuda()
        
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        model.load_state_dict(torch.load(modelPath),strict=False)
    else:
        state_dict = torch.load(modelPath)
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:]
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)  
        
    model.eval()
    
    ynum = localPatch.shape[0]
    xnum = localPatch.shape[1]
    scal = localPatch.shape[2]
    
    totalFlow = np.zeros((ynum, xnum, 2, scal, scal), dtype = np.float32)
    
    for j in range(0, ynum):
        for i in range(0, xnum):

            temp_localPatch = localPatch[j, i]
            temp_globaPatch = globalPatch[j, i]
        
            temp_localPatch = transform(temp_localPatch)
            temp_globaPatch = transform(temp_globaPatch)

            if torch.cuda.is_available():
                temp_localPatch = temp_localPatch.cuda()
                temp_globaPatch = temp_globaPatch.cuda()

            temp_localPatch = temp_localPatch.view(1,3,scal,scal)
            temp_globaPatch = temp_globaPatch.view(1,3,256,256)
            
            temp_localPatch = Variable(temp_localPatch)
            temp_globaPatch = Variable(temp_globaPatch)
            
            flow_output = model(temp_localPatch, temp_globaPatch)

            u = flow_output.data.cpu().numpy()[0][0]
            v = flow_output.data.cpu().numpy()[0][1]
            
            totalFlow[j,i,0] = u
            totalFlow[j,i,1] = v

    return totalFlow

In [77]:
totalFlow = testRealFlow(modelPath, totalLocalPatch, totalGlobaPatch)

In [79]:
np.save(saveFlowPath, totalFlow)

In [27]:
# python eval.py [--imgPath [PATH]] [--modelPath [PATH]]
#                [--saveImgPath [PATH]] [--saveFlowPath [PATH]]

In [78]:
totalFlow.shape

(6, 3, 2, 256, 256)

In [86]:
totalFlow[0][0].shape

(2, 256, 256)

In [30]:
import numpy as np
import skimage.io as io
from numba import cuda
import math
import argparse

In [40]:
img_path = 'Wrap.jpg'
flow_path = 'IMG_WRAP.npy'

@cuda.jit(device=True)
def iterSearchShader(padu, padv, xr, yr, maxIter, precision):
    
    H = padu.shape[0] - 1
    W = padu.shape[1] - 1
    
    if abs(padu[yr,xr]) < precision and abs(padv[yr,xr]) < precision:
        return xr, yr

    else:
        # Our initialize method in this paper, can see the overleaf for detail
        if (xr + 1) <= (W - 1):
            dif = padu[yr,xr + 1] - padu[yr,xr]
            u_next = padu[yr,xr]/(1 + dif)
        else:
            dif = padu[yr,xr] - padu[yr,xr - 1]
            u_next = padu[yr,xr]/(1 + dif)

        if (yr + 1) <= (H - 1):
            dif = padv[yr + 1,xr] - padv[yr,xr]
            v_next = padv[yr,xr]/(1 + dif)
        else:
            dif = padv[yr,xr] - padv[yr - 1,xr]
            v_next = padv[yr,xr]/(1 + dif)

        i = xr - u_next
        j = yr - v_next
        '''
        i = xr - padu[yr,xr]
        j = yr - padv[yr,xr]
        '''
        # The same as traditinal iterative search method
        for iter in range(maxIter):

            if 0<= i <= (W - 1) and 0 <= j <= (H - 1):

                u11 = padu[int(j), int(i)]
                v11 = padv[int(j), int(i)]

                u12 = padu[int(j), int(i) + 1]
                v12 = padv[int(j), int(i) + 1]

                u21 = padu[int(j) + 1, int(i)]
                v21 = padv[int(j) + 1, int(i)]

                u22 = padu[int(j) + 1, int(i) + 1]
                v22 = padv[int(j) + 1, int(i) + 1]


                u = u11*(int(i) + 1 - i)*(int(j) + 1 - j) + u12*(i - int(i))*(int(j) + 1 - j) + \
                    u21*(int(i) + 1 - i)*(j - int(j)) + u22*(i - int(i))*(j - int(j))

                v = v11*(int(i) + 1 - i)*(int(j) + 1 - j) + v12*(i - int(i))*(int(j) + 1 - j) + \
                    v21*(int(i) + 1 - i)*(j - int(j)) + v22*(i - int(i))*(j - int(j))

                i_next = xr - u
                j_next = yr - v                

                if abs(i - i_next)<precision and abs(j - j_next)<precision:
                    return i, j

                i = i_next
                j = j_next

            else:     
                return -1, -1
        '''
        return -1, -1
        '''
        # if the search doesn't converge within max iter, it will return the last iter result
        if 0 <= i_next <= (W - 1) and 0 <= j_next <= (H - 1):
            return i_next, j_next
        elif 0 <= i <= (W - 1) and 0 <= j <= (H - 1):
            return i, j
        else:
            return -1, -1
        

            
@cuda.jit(device=True)
def biInterpolation(distorted, i, j):
    Q11 = distorted[int(j), int(i)]
    Q12 = distorted[int(j), int(i) + 1]
    Q21 = distorted[int(j) + 1, int(i)]
    Q22 = distorted[int(j) + 1, int(i) + 1]
    pixel = Q11*(int(i) + 1 - i)*(int(j) + 1 - j) + Q12*(i - int(i))*(int(j) + 1 - j) + \
            Q21*(int(i) + 1 - i)*(j - int(j)) + Q22*(i - int(i))*(j - int(j))
    return pixel


@cuda.jit
def iterSearch(padu, padv, paddistorted, resultImg, maxIter, precision, resultMsk):
    
    H = padu.shape[0] - 1
    W = padu.shape[1] - 1
    
    start_x, start_y = cuda.grid(2)
    stride_x, stride_y = cuda.gridsize(2)
    
    for xr in range(start_x, W, stride_x):
        for yr in range(start_y, H, stride_y):

            i,j = iterSearchShader(padu, padv, xr, yr, maxIter, precision)

            if(i != -1) and (j != -1):
                resultImg[yr, xr,0] = biInterpolation(paddistorted[:,:,0], i, j)
                resultImg[yr, xr,1] = biInterpolation(paddistorted[:,:,1], i, j)
                resultImg[yr, xr,2] = biInterpolation(paddistorted[:,:,2], i, j)
            else:
                resultMsk[yr, xr] = 255

@cuda.jit
def iterSearchGrey(padu, padv, paddistorted, resultImg, maxIter, precision, resultMsk):
    
    H = padu.shape[0] - 1
    W = padu.shape[1] - 1
  
    start_x, start_y = cuda.grid(2)
    stride_x, stride_y = cuda.gridsize(2)
    
    for xr in range(start_x, W, stride_x):
        for yr in range(start_y, H, stride_y):

            i,j = iterSearchShader(padu, padv, xr, yr, maxIter, precision)

            if(i != -1) and (j != -1):
                resultImg[yr, xr] = biInterpolation(paddistorted[:,:], i, j)
            else:
                resultMsk[yr, xr] = 255

def rectification(distorted, flow):
    
    H = distorted.shape[0]
    W = distorted.shape[1]

    maxIter = 100
    precision = 1e-2

    isGrey = True
    resultMsk = np.array(np.zeros((H, W)), dtype = np.uint8)
    if len(distorted.shape) == 3:
        resultImg = np.array(np.zeros((H, W, 3)), dtype = np.uint8)
        paddistorted = np.array(np.zeros((H + 1, W + 1, 3)), dtype = np.uint8)
        resultImg.fill(255)
        isGrey = False
    else:
        resultImg = np.array(np.zeros((H, W)), dtype = np.uint8)
        paddistorted = np.array(np.zeros((H + 1, W + 1)), dtype = np.uint8)
        resultImg.fill(255)
        isGrey = True

    paddistorted[0:H, 0:W] = distorted[0:H, 0:W]
    paddistorted[H, 0:W] = distorted[H-1, 0:W]
    paddistorted[0:H, W] = distorted[0:H, W-1]
    paddistorted[H, W] = distorted[H-1, W-1]

    padu = np.array(np.zeros((H + 1, W + 1)), dtype = np.float32)
    padu[0:H, 0:W] = flow[0][0:H, 0:W]
    padu[H, 0:W] = flow[0][H-1, 0:W]
    padu[0:H, W] = flow[0][0:H, W-1]
    padu[H, W] = flow[0][H-1, W-1]

    padv = np.array(np.zeros((H + 1, W + 1)), dtype = np.float32)
    padv[0:H, 0:W] = flow[1][0:H, 0:W]
    padv[H, 0:W] = flow[1][H-1, 0:W]
    padv[0:H, W] = flow[1][0:H, W-1]
    padv[H, W] = flow[1][H-1, W-1]

    padu = cuda.to_device(padu)
    padv = cuda.to_device(padv)
    paddistorted = cuda.to_device(paddistorted)
    resultImg = cuda.to_device(resultImg)
    resultMsk = cuda.to_device(resultMsk)

    threadsperblock = (16, 16)
    blockspergrid_x = math.ceil(W / threadsperblock[0])
    blockspergrid_y = math.ceil(H / threadsperblock[1])
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    if isGrey:
        iterSearchGrey[blockspergrid, threadsperblock](padu, padv, paddistorted, resultImg, maxIter, precision, resultMsk)
    else:
        iterSearch[blockspergrid, threadsperblock](padu, padv, paddistorted, resultImg, maxIter, precision, resultMsk)

    resultImg = resultImg.copy_to_host()
    resultMsk = resultMsk.copy_to_host()
    
    return resultImg, resultMsk
    
distortedImg = io.imread('Wrap.jpg')  
flow = np.load(flow_path)
resImg, resMsk = rectification(distortedImg, flow)
io.imsave('result.png', resImg) 

ValueError: could not broadcast input array from shape (3,2,256,256) into shape (1024,768)

In [56]:
distorted = io.imread('result.jpg')  

In [57]:
H = distorted.shape[0]
W = distorted.shape[1]

In [58]:
H

2000

In [59]:
W

1500

In [60]:
    maxIter = 100
    precision = 1e-2

    isGrey = True

In [61]:
if len(distorted.shape) == 3:
        resultImg = np.array(np.zeros((H, W, 3)), dtype = np.uint8)
        paddistorted = np.array(np.zeros((H + 1, W + 1, 3)), dtype = np.uint8)
        resultImg.fill(255)
        isGrey = False
else:
    resultImg = np.array(np.zeros((H, W)), dtype = np.uint8)
    paddistorted = np.array(np.zeros((H + 1, W + 1)), dtype = np.uint8)
    resultImg.fill(255)
    isGrey = True

In [62]:
paddistorted[0:H, 0:W] = distorted[0:H, 0:W]
paddistorted[H, 0:W] = distorted[H-1, 0:W]
paddistorted[0:H, W] = distorted[0:H, W-1]
paddistorted[H, W] = distorted[H-1, W-1]

In [63]:
flow = np.load(flow_path)

In [64]:
padu = np.array(np.zeros((H + 1, W + 1)), dtype = np.float32)
padu[0:H, 0:W] = flow[0][0:H, 0:W]
padu[H, 0:W] = flow[0][H-1, 0:W]
padu[0:H, W] = flow[0][0:H, W-1]
padu[H, W] = flow[0][H-1, W-1]

ValueError: could not broadcast input array from shape (3,2,256,256) into shape (2000,1500)

In [67]:
padu[:H,:W].shape

(2000, 1500)

In [69]:
flow[0].shape

(3, 2, 256, 256)